In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing Libraries

In [2]:
import opendatasets as od
import os
import numpy as np
import cv2
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras import Sequential

Downloading the dataset

In [3]:
od.download('https://www.kaggle.com/datasets/salader/dogs-vs-cats')

Skipping, found downloaded files in "./dogs-vs-cats" (use force=True to force download)


Preparing the Model

In [4]:
base_model = VGG16(include_top = False,weights='imagenet',input_shape=(150,150,3))

In [5]:
from keras.layers import Flatten
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation = 'sigmoid'))
base_model.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 128)               1048704   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15,763,521
Trainable params: 1,048,833
Non-trainable params: 14,714,688
_________________________________________________________________


Preparing the training dataset and labels

In [6]:
data_path = '/content/dogs-vs-cats/train/'
class_names = sorted(os.listdir(data_path))
num_classes = len(class_names)
print(class_names)

['cats', 'dogs']


In [7]:
from keras.applications.vgg16 import preprocess_input
train_data = np.empty((6000,150,150,3))
i=0
for cl in class_names:
  k=0
  print(cl,end = '=>')
  for img in os.listdir(data_path + cl):
        if(k==3000):
            break
        image = np.asarray(cv2.resize(cv2.imread(data_path+cl+'/'+img,cv2.IMREAD_COLOR),[150,150]))
        image = preprocess_input(image)
        train_data[i]=image
        i=i+1
        k=k+1
  print('done')

cats=>done
dogs=>done


In [8]:
labels = np.empty(6000,dtype = int)
labels[:3000] = 0
labels[3000:] = 1

Preparing the validation data and labels

In [9]:
valid_data = np.empty((1000,150,150,3))
data_path = '/content/dogs-vs-cats/test/'
class_name = sorted(os.listdir(data_path))
num_classes = len(class_name)
print(class_name)

['cats', 'dogs']


In [10]:
i=0
for cl in class_name:
  k=0
  print(cl,end = '=>')
  for img in os.listdir(data_path + cl):
        if(k==500):
            break
        image = np.asarray(cv2.resize(cv2.imread(data_path+cl+'/'+img,cv2.IMREAD_COLOR),[150,150]))
        image = preprocess_input(image)
        valid_data[i]=image
        i=i+1
        k=k+1
  print('done')

cats=>done
dogs=>done


In [11]:
valid_labels = np.empty(1000,dtype=int)
valid_labels[:500] = 0
valid_labels[500:] = 1

Compiling and fitting the model

In [12]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [13]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss',patience=3)


In [14]:
model.fit(x=train_data,y=labels,epochs=20,batch_size = 32,validation_data=(valid_data,valid_labels),callbacks=[callback])

Epoch 1/20
188/188 [==============================] - 34s 106ms/step - loss: 0.9541 - accuracy: 0.9197 - val_loss: 0.2012 - val_accuracy: 0.9370
Epoch 2/20
188/188 [==============================] - 15s 82ms/step - loss: 0.0677 - accuracy: 0.9728 - val_loss: 0.1705 - val_accuracy: 0.9480
Epoch 3/20
188/188 [==============================] - 16s 84ms/step - loss: 0.0293 - accuracy: 0.9880 - val_loss: 0.2401 - val_accuracy: 0.9470
Epoch 4/20
188/188 [==============================] - 16s 85ms/step - loss: 0.0163 - accuracy: 0.9923 - val_loss: 0.2224 - val_accuracy: 0.9440
Epoch 5/20
188/188 [==============================] - 16s 85ms/step - loss: 0.0120 - accuracy: 0.9960 - val_loss: 0.2637 - val_accuracy: 0.9460


In [15]:
train_loss, train_acc = model.evaluate(x=train_data,y=labels)
test_loss, test_acc   = model.evaluate(x=valid_data,y=valid_labels)
print("final train accuracy = {:.2f} , Test accuracy = {:.2f}".format(train_acc*100, test_acc*100))

32/32 [==============================] - 2s 68ms/step - loss: 0.2637 - accuracy: 0.9460
final train accuracy = 99.73 , Test accuracy = 94.60
